In [20]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class PuzzleModel(nn.Module):
    def __init__(self, input_size):
        super(PuzzleModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

model = PuzzleModel(256)
checkpoint_path = '/kaggle/input/heuristic/puzzle_model.pth'
checkpoint = torch.load(checkpoint_path)
if "module." in list(checkpoint.keys())[0]:
    new_state_dict = {}
    for key, value in checkpoint.items():
        new_key = key.replace("module.", "")
        new_state_dict[new_key] = value
    checkpoint = new_state_dict
model.load_state_dict(checkpoint)
model.eval()

input_data = torch.tensor([10,1,5,4,14,11,15,7,0,9,3,2,6,8,12,13], dtype=torch.float32)
input_data_one_hot = torch.tensor(np.eye(16)[input_data.to(torch.int64)].ravel(), dtype=torch.float32)
with torch.no_grad():
    output = model(input_data_one_hot)
    predicted_value = output.item()

print("Predicted Value:", predicted_value)

Predicted Value: 47.89495849609375
